In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures 
from urllib.request import urlopen
import time
import threading

URLS = [
    "http://www.baidu.com",
    "http://www.baidu.com",
    "http://www.baidu.com",
    "http://www.baidu.com",
    "http://www.baidu.com"
]

# 全局变量
program_start = None
task_counter = 0
counter_lock = threading.Lock()

def load_url(url, timeout):
     global task_counter, program_start
     with counter_lock:
         task_counter += 1
         task_id = task_counter
     
     start_time = time.time()
     elapsed_from_start = start_time - program_start
     thread_id = threading.current_thread().name
     
     print(f"[{elapsed_from_start:.3f}s] 任务{task_id} 开始执行 | 线程: {thread_id} | URL: {url}")
     
     with urlopen(url, timeout=timeout) as conn:
         try:
            result = conn.read()
            end_time = time.time()
            elapsed = end_time - start_time
            elapsed_from_start_end = end_time - program_start
            print(f"[{elapsed_from_start_end:.3f}s] 任务{task_id} 完成 | 线程: {thread_id} | 耗时: {elapsed:.2f}秒 | 大小: {len(result)} bytes")
            return result
         except Exception as e:
             print(f"[{time.time() - program_start:.3f}s] 任务{task_id} 出错: {e}")
             return None


program_start = time.time()
task_counter = 0

print(f"[0.000s] 所有任务已提交（max_workers=2，同时只执行2个线程）\n")

with ThreadPoolExecutor(max_workers=1) as executor:
     futres_url = {
         executor.submit(load_url, url, 60): url for url in URLS
     }

     for future in concurrent.futures.as_completed(futres_url):
          url = futres_url[future]
          try:
            data = future.result()
          except Exception as e:
            print('[%.3fs] %r generated an exception: %s' % (time.time() - program_start, url, e))

total_time = time.time() - program_start
print(f"\n[总计] 所有任务完成，总耗时: {total_time:.2f} 秒")  


[0.000s] 所有任务已提交（max_workers=2，同时只执行2个线程）

[0.001s] 任务1 开始执行 | 线程: ThreadPoolExecutor-10_0 | URL: http://www.baidu.com
[0.117s] 任务1 完成 | 线程: ThreadPoolExecutor-10_0 | 耗时: 0.12秒 | 大小: 29506 bytes
[0.117s] 任务2 开始执行 | 线程: ThreadPoolExecutor-10_0 | URL: http://www.baidu.com
[0.213s] 任务2 完成 | 线程: ThreadPoolExecutor-10_0 | 耗时: 0.10秒 | 大小: 29506 bytes
[0.214s] 任务3 开始执行 | 线程: ThreadPoolExecutor-10_0 | URL: http://www.baidu.com
[0.314s] 任务3 完成 | 线程: ThreadPoolExecutor-10_0 | 耗时: 0.10秒 | 大小: 29506 bytes
[0.314s] 任务4 开始执行 | 线程: ThreadPoolExecutor-10_0 | URL: http://www.baidu.com
[0.404s] 任务4 完成 | 线程: ThreadPoolExecutor-10_0 | 耗时: 0.09秒 | 大小: 29506 bytes
[0.404s] 任务5 开始执行 | 线程: ThreadPoolExecutor-10_0 | URL: http://www.baidu.com
[0.498s] 任务5 完成 | 线程: ThreadPoolExecutor-10_0 | 耗时: 0.09秒 | 大小: 29506 bytes

[总计] 所有任务完成，总耗时: 0.50 秒
